In [ ]:
!pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmprcxvf4ky".


For reference

In [ ]:
%%cuda -c "--gpu-architecture sm_75 -O2 --default-stream per-thread"
#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <cuda_runtime.h>

__global__ void testKernel(float*x, int len)
{
	int tid = threadIdx.x + blockIdx.x * blockDim.x;

	if(tid < len) {
		float sum = x[tid];
		int iter = 0;

		while(iter++ < len) {
			sum += 1;
		}
		x[tid] = sum;
	}

}


__global__ void subKernel (float*a, float*b, float*c, int len)
{
	int tid = threadIdx.x + blockIdx.x * blockDim.x;

	if(tid < len) {
		c[tid] = a[tid] - b[tid];
	}

}


int main(int argc, char **argv)
{
	const int streamsNum = 2;

	int N=1<<10; // 1Kibi

	std::cout << "Running " << N << " (floats) as the input data size." << std::endl;
	std::cout << "Launching " << streamsNum << " cuda streams." << std::endl;

	// host
	float *h_a, *h_b, *h_c;
	cudaMallocHost((void**)&h_a, sizeof(float) * N);
	cudaMallocHost((void**)&h_b, sizeof(float) * N);
	cudaMallocHost((void**)&h_c, sizeof(float) * N);

	cudaMemset(h_a, 0, sizeof(float) * N);
	cudaMemset(h_b, 0, sizeof(float) * N);

	// device
	float *d_a, *d_b, *d_c;

	cudaMalloc((void**)&d_a, sizeof(float) * N);
	cudaMalloc((void**)&d_b, sizeof(float) * N);
	cudaMalloc((void**)&d_c, sizeof(float) * N);

	// streams
	cudaStream_t streams[streamsNum];
	cudaEvent_t  events[streamsNum]; // events for streams

	for(int i=0; i<streamsNum; i++) {
		cudaStreamCreate(&streams[i]);
		cudaEventCreate(&events[i]);
	}

	// h2d
	cudaMemcpyAsync(d_a, h_a, sizeof(float)*N, cudaMemcpyHostToDevice, streams[0]);
	cudaMemcpyAsync(d_b, h_b, sizeof(float)*N, cudaMemcpyHostToDevice, streams[1]);

	// kernel
	dim3 block = dim3(128,1,1);
	dim3 grid = dim3((N + block.x - 1) / block.x,1,1);

	testKernel <<< grid, block, 0, streams[0] >>> (d_a, N); // a + x
	cudaEventRecord(events[0], streams[0]);
	testKernel <<< grid, block, 0, streams[1] >>> (d_b, N); // b + x
	cudaEventRecord(events[1], streams[1]);

	cudaEventSynchronize(events[0]);
	cudaEventSynchronize(events[1]);

	subKernel <<< grid, block, 0, streams[0] >>> (d_a, d_b, d_c, N); // a - b

	// d2h
	cudaMemcpyAsync(h_c, d_c, sizeof(float)*N, cudaMemcpyDeviceToHost, streams[0]);

	cudaDeviceSynchronize(); // NOTE: this is needed to make sure prev dev opt is done!

	int error_c = 0;
	for(int i=0; i<N; i++) {
		if(h_c[i] > 1e-8) {  // h_c should be 0
			printf("h_c[%d] = %f\n",i, h_c[i]);
			error_c += 1;
		}
	}
	if(error_c == 0) {
		printf("Pass test on h_c!\n");
	}


	// free
	for(int i=0; i<streamsNum; i++) {
		cudaStreamDestroy(streams[i]);
		cudaEventDestroy(events[i]);
	}

	cudaFree(d_a);
	cudaFree(d_b);
	cudaFree(d_c);

	cudaFreeHost(h_a);
	cudaFreeHost(h_b);
	cudaFreeHost(h_c);

	return 0;
}

In [ ]:
%%cuda -c "--gpu-architecture sm_75 -O2 --default-stream per-thread"
#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <cuda_runtime.h>

__global__ void testKernel(float*x, int len)
{
	int tid = threadIdx.x + blockIdx.x * blockDim.x;

	if(tid < len) {
		float sum = x[tid];
		int iter = 0;

		while(iter++ < len) {
			sum += 1;
		}
		x[tid] = sum;
	}

}


__global__ void subKernel (float*a, float*b, float*c, int len)
{
	int tid = threadIdx.x + blockIdx.x * blockDim.x;

	if(tid < len) {
		c[tid] = a[tid] - b[tid];
	}

}


int main(int argc, char **argv)
{
	const int streamsNum = 2;

	int N=1<<10; // 1Kibi

	std::cout << "Running " << N << " (floats) as the input data size." << std::endl;
	std::cout << "Launching " << streamsNum << " cuda streams." << std::endl;

	// host
	float *h_a, *h_b, *h_c;
	cudaMallocHost((void**)&h_a, sizeof(float) * N);
	cudaMallocHost((void**)&h_b, sizeof(float) * N);
	cudaMallocHost((void**)&h_c, sizeof(float) * N);

	cudaMemset(h_a, 0, sizeof(float) * N);
	cudaMemset(h_b, 0, sizeof(float) * N);

	// device
	float *d_a, *d_b, *d_c;

	cudaMalloc((void**)&d_a, sizeof(float) * N);
	cudaMalloc((void**)&d_b, sizeof(float) * N);
	cudaMalloc((void**)&d_c, sizeof(float) * N);

	// streams
	cudaStream_t streams[streamsNum];
	cudaEvent_t  events[streamsNum]; // events for streams

	for(int i=0; i<streamsNum; i++) {
		cudaStreamCreate(&streams[i]);
		cudaEventCreate(&events[i]);
	}

	// h2d
	cudaMemcpyAsync(d_a, h_a, sizeof(float)*N, cudaMemcpyHostToDevice, streams[0]);
	cudaMemcpyAsync(d_b, h_b, sizeof(float)*N, cudaMemcpyHostToDevice, streams[1]);

	// kernel
	dim3 block = dim3(128,1,1);
	dim3 grid = dim3((N + block.x - 1) / block.x,1,1);

	testKernel <<< grid, block, 0, streams[0] >>> (d_a, N); // a + x
	cudaEventRecord(events[0], streams[0]);
	testKernel <<< grid, block, 0, streams[1] >>> (d_b, N); // b + x
	cudaEventRecord(events[1], streams[1]);

	cudaEventSynchronize(events[0]);
	cudaEventSynchronize(events[1]);

	subKernel <<< grid, block, 0, streams[0] >>> (d_a, d_b, d_c, N); // a - b

	// d2h
	cudaMemcpyAsync(h_c, d_c, sizeof(float)*N, cudaMemcpyDeviceToHost, streams[0]);

	cudaDeviceSynchronize(); // NOTE: this is needed to make sure prev dev opt is done!

	int error_c = 0;
	for(int i=0; i<N; i++) {
		if(h_c[i] > 1e-8) {  // h_c should be 0
			printf("h_c[%d] = %f\n",i, h_c[i]);
			error_c += 1;
		}
	}
	if(error_c == 0) {
		printf("Pass test on h_c!\n");
	}


	// free
	for(int i=0; i<streamsNum; i++) {
		cudaStreamDestroy(streams[i]);
		cudaEventDestroy(events[i]);
	}

	cudaFree(d_a);
	cudaFree(d_b);
	cudaFree(d_c);

	cudaFreeHost(h_a);
	cudaFreeHost(h_b);
	cudaFreeHost(h_c);

	return 0;
}

Running 1024 (floats) as the input data size.
Launching 2 cuda streams.
Pass test on h_c!



In [ ]:
%%cuda -c "--gpu-architecture sm_75 -O2 --default-stream per-thread"
#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <cuda_runtime.h>

__global__ void testKernel(float*x, int len)
{
	int tid = threadIdx.x + blockIdx.x * blockDim.x;

	if(tid < len) {
		float sum = x[tid];
		int iter = 0;

		while(iter++ < len) {
			sum += 1;
		}
		x[tid] = sum;
	}

}


__global__ void subKernel (float*a, float*b, float*c, int len)
{
	int tid = threadIdx.x + blockIdx.x * blockDim.x;

	if(tid < len) {
		c[tid] = a[tid] - b[tid];
	}

}


int main(int argc, char **argv)
{
	const int streamsNum = 2;

	int N=1<<12; // 1Kibi

	std::cout << "Running " << N << " (floats) as the input data size." << std::endl;
	std::cout << "Launching " << streamsNum << " cuda streams." << std::endl;

	// host
	float *h_a, *h_b, *h_c;
	cudaMallocHost((void**)&h_a, sizeof(float) * N);
	cudaMallocHost((void**)&h_b, sizeof(float) * N);
	cudaMallocHost((void**)&h_c, sizeof(float) * N);

	cudaMemset(h_a, 0, sizeof(float) * N);
	cudaMemset(h_b, 0, sizeof(float) * N);

	// device
	float *d_a, *d_b, *d_c;

	cudaMalloc((void**)&d_a, sizeof(float) * N);
	cudaMalloc((void**)&d_b, sizeof(float) * N);
	cudaMalloc((void**)&d_c, sizeof(float) * N);

	// streams
	cudaStream_t streams[streamsNum];
	//cudaEvent_t  events[streamsNum]; // events for streams

	for(int i=0; i<streamsNum; i++) {
		cudaStreamCreate(&streams[i]);
		//cudaEventCreate(&events[i]);
	}

	// h2d
	cudaMemcpyAsync(d_a, h_a, sizeof(float)*N, cudaMemcpyHostToDevice, streams[0]);
	cudaMemcpyAsync(d_b, h_b, sizeof(float)*N, cudaMemcpyHostToDevice, streams[1]);

	// kernel
	dim3 block = dim3(128,1,1);
	dim3 grid = dim3((N + block.x - 1) / block.x,1,1);

	testKernel <<< grid, block, 0, streams[0] >>> (d_a, N); // a + x
	//cudaEventRecord(events[0], streams[0]);
	testKernel <<< grid, block, 0, streams[1] >>> (d_b, N); // b + x
	//cudaEventRecord(events[1], streams[1]);

	//cudaEventSynchronize(events[0]);
	//cudaEventSynchronize(events[1]);

	subKernel <<< grid, block, 0, streams[0] >>> (d_a, d_b, d_c, N); // a - b

	// d2h
	cudaMemcpyAsync(h_c, d_c, sizeof(float)*N, cudaMemcpyDeviceToHost, streams[0]);

	//cudaDeviceSynchronize(); // NOTE: this is needed to make sure prev dev opt is done!

	int error_c = 0;
	for(int i=0; i<N; i++) {
		if(h_c[i] > 1e-8) {  // h_c should be 0
			printf("h_c[%d] = %f\n",i, h_c[i]);
			error_c += 1;
		}
	}
	if(error_c == 0) {
		printf("Pass test on h_c!\n");
	}


	// free
	for(int i=0; i<streamsNum; i++) {
		cudaStreamDestroy(streams[i]);
		//cudaEventDestroy(events[i]);
	}

	cudaFree(d_a);
	cudaFree(d_b);
	cudaFree(d_c);

	cudaFreeHost(h_a);
	cudaFreeHost(h_b);
	cudaFreeHost(h_c);

	return 0;
}

Running 4096 (floats) as the input data size.
Launching 2 cuda streams.
Pass test on h_c!



In [ ]:
%%cuda -c "--gpu-architecture sm_75 -O2 --default-stream per-thread"
#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <cuda_runtime.h>

// Kernel with a heavy workload (more iterations)
__global__ void testKernel(float* x, int len)
{
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid < len) {
        float sum = x[tid];
        // Increase iterations to force longer execution time
        int iter = 0;
        while(iter++ < len * 100) {  // increased workload factor (100x)
            sum += 1;
        }
        x[tid] = sum;
    }
}

__global__ void subKernel (float* a, float* b, float* c, int len)
{
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if(tid < len) {
        c[tid] = a[tid] - b[tid];
    }
}

int main(int argc, char **argv)
{
    const int streamsNum = 2;

    // Use a larger N to further delay kernel execution
    int N = 1 << 20; // 1M elements

    std::cout << "Running " << N << " (floats) as the input data size." << std::endl;
    std::cout << "Launching " << streamsNum << " cuda streams." << std::endl;

    // Allocate pinned host memory
    float *h_a, *h_b, *h_c;
    cudaMallocHost((void**)&h_a, sizeof(float) * N);
    cudaMallocHost((void**)&h_b, sizeof(float) * N);
    cudaMallocHost((void**)&h_c, sizeof(float) * N);

    cudaMemset(h_a, 0, sizeof(float) * N);
    cudaMemset(h_b, 0, sizeof(float) * N);

    // Allocate device memory
    float *d_a, *d_b, *d_c;
    cudaMalloc((void**)&d_a, sizeof(float) * N);
    cudaMalloc((void**)&d_b, sizeof(float) * N);
    cudaMalloc((void**)&d_c, sizeof(float) * N);

    // Create two streams
    cudaStream_t streams[streamsNum];
    for(int i=0; i<streamsNum; i++) {
        cudaStreamCreate(&streams[i]);
    }

    // Asynchronously copy from host to device
    cudaMemcpyAsync(d_a, h_a, sizeof(float)*N, cudaMemcpyHostToDevice, streams[0]);
    cudaMemcpyAsync(d_b, h_b, sizeof(float)*N, cudaMemcpyHostToDevice, streams[1]);

    // Set up execution configuration
    dim3 block(128, 1, 1);
    dim3 grid((N + block.x - 1) / block.x, 1, 1);

    // Launch kernels in separate streams
    testKernel<<< grid, block, 0, streams[0] >>>(d_a, N);
    testKernel<<< grid, block, 0, streams[1] >>>(d_b, N);

    // Launch subtraction kernel in stream[0]
    subKernel<<< grid, block, 0, streams[0] >>>(d_a, d_b, d_c, N);

    // Asynchronously copy result back to host in stream[0]
    cudaMemcpyAsync(h_c, d_c, sizeof(float)*N, cudaMemcpyDeviceToHost, streams[0]);

    // No synchronization! Immediately access h_c on the host.
    // This loop may access h_c before the device work is finished.
    int error_c = 0;
    for(int i = 0; i < N; i++) {
        if (h_c[i] > 1e-8) {  // Expected: h_c should be 0
            printf("h_c[%d] = %f\n", i, h_c[i]);
            error_c++;
        }
    }
    if(error_c == 0) {
        printf("Pass test on h_c! (but this is by chance without sync)\n");
    } else {
        printf("Test failed: detected %d errors in h_c (race condition)!\n", error_c);
    }

    // Clean up: destroy streams and free memory
    for(int i=0; i<streamsNum; i++) {
        cudaStreamDestroy(streams[i]);
    }
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    cudaFreeHost(h_a);
    cudaFreeHost(h_b);
    cudaFreeHost(h_c);

    return 0;
}


Running 1048576 (floats) as the input data size.
Launching 2 cuda streams.
Pass test on h_c! (but this is by chance without sync)

